In [1]:
import os
import time
import pandas as pd
import json
import ast
import pprint as pp
from tabulate import tabulate

#also needs pip install: GPT4All, chromadb
from langchain.llms import Ollama

#import text loaders
from langchain.docstore.document import Document
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_community.document_loaders import DataFrameLoader
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import UnstructuredFileLoader

import langchain_community.vectorstores.utils as vutils
from langchain.prompts import PromptTemplate
from langchain.chains.llm import LLMChain
from langchain.output_parsers import PandasDataFrameOutputParser

#import character splitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

#for storing embeddings
from langchain.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma

# for query
from langchain.chains import RetrievalQA

/tmp/ipykernel_4929/299398804.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Single Test Case
ollama = Ollama(base_url='http://localhost:11434', model='llama2:13b', format='json')
#Greatest success rate found for correct formatting used ollama model llama2:13b
#Other models such as llama2:70b or llama2-uncensored may have a higher success rate
#
filepath = "/home/team-2/ExtractedEmails/PRO_EDR_ Cholera, diarrhea _ dysentery update (04)_ Africa (Zambia, Sudan).eml.txt"
test = open(filepath, "r")
text = test.readlines()
print(text)
response  = ollama('[INST] Give me the name of the microorganism, type of microorganism (bateria, virus, fungus, etc.), is it a bloodborne pathogen or sexually transmitted disease or a pathogen that only affects animals or a pathogen that only affects plants, the country of origin, and the type of outbreak (Limited, Sporadic, Multinational, National, Pandemic) of the pathogen described in this text {0}. [/INST] Use this format: {{"Name_Of_Disease":"African Swine Fever (ASF)", "Type_Of_Microorganism":"Virus(E)", "Human_Only":"false", "Area_Of_Origin":"Mexico, “Type_Of_Outbreak":“Pandemic” }}[/INST]Do not follow up with any text [/INST]If you do not know the answer just say unknown'.format(text))
test.close()

print(response)


['CHOLERA, DIARRHEA & DYSENTERY UPDATE (04): AFRICA (ZAMBIA, SUDAN)\n', '*****************************************************************\n', 'A ProMED-mail post\n', '<http://www.promedmail.org>\n', 'ProMED-mail is a program of the\n', 'International Society for Infectious Diseases\n', '<http://www.isid.org>\n', '\n', 'In this update:\n', '[1] Zambia\n', '[2] Sudan\n', '\n', '******\n', '[1] Zambia\n', 'Date: Sat 20 Jan 2024\n', 'Source: APO Group - Africa Newsroom [edited]\n', '<https://www.africa-newsroom.com/press/zambia-races-to-curb-fastspreading-c=\n', 'holera-outbreak?lang=3Den>\n', '\n', '\n', 'As the cholera outbreak in Zambia continues to expand rapidly, efforts\n', 'are being stepped up to halt the spread of disease and save lives. A\n', 'vaccination campaign is currently underway targeting 1.5 million\n', 'people, including children, health workers, and people at high risk of\n', 'infection in the worst affected areas.\n', '\n', 'In partnership with Gavi, the Vaccine Allia

/home/team-2/.local/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


{"Name_Of_Disease": "Cholera", "Type_Of_Microorganism": "Bacteria (Vibrio cholerae)", "Human_Only": "true", "Area_Of_Origin": "Zambia and Sudan", "Type_Of_Outbreak": "Multinational" }


In [3]:
#Single test case to classify which business unit might be interested
medOllama = Ollama(base_url='http://localhost:11434', model='llama2:13b', format='json')
buResponse  = medOllama('[INST] Given the Type_Of_Microorganism in {0}, return the key of what business unit(s) may be interesed ' +
'[/INST] P&Gs 6 business units are: {{"FHC":"Relates to pathogens transmitted via contaminated surfaces, "' +
'"PHC":"Upper respiratory tract infections, "' +
'"OC": "Oral Cavity Pathogens & Upper respiratory tract infections, "' +
'"PCC": "Pathogens transmitted via poor hand hygiene, "' +
'"PGV": "Pathogens transmitted via insects/arthropods, "' +
'"BFF": "Pathogens transmitted via bodily excrements and can linger in diapers or feminine pads"}}' + 
' [/INST Do not follow up with any text'.format(response))
print(buResponse)

{ "PHC": "Upper respiratory tract infections" }


In [4]:
ollama = Ollama(base_url='http://localhost:11434', model='llama2:13b', format='json')
EmailDirectory = "/home/team-2/ExtractedEmails"
files = os.listdir(EmailDirectory)
responses = []
index = 0
for email in files:
    test = open(EmailDirectory + "/" + email, "r")
    text = test.readlines()
    response  = ollama('[INST] Give me the name of the microorganism, type of microorganism (bateria, virus, fungus, etc.), is it a bloodborne pathogen or sexually transmitted disease or a pathogen that only affects animals or a pathogen that only affects plants, the country of origin, and the type of outbreak (Limited, Sporadic, Multinational, National, Pandemic) of the pathogen described in this text {0}. [/INST] Use this format: {{"Name_Of_Disease":"African Swine Fever (ASF)", "Type_Of_Microorganism":"Virus(E)", "Human_Only":"false", "Area_Of_Origin":"Mexico, “Type_Of_Outbreak":“Pandemic” }}[/INST]Do not follow up with any text [/INST]If you do not know the answer just say unknown'.format(text))
    test.close()
    responses.append(response + '\n')
    time.sleep(.1)
    index += 1
    print(index)

# index here is unimportant, it is just used to gain a visualization of where the program
# is at in the loop and if it is stuck
print(responses)

1
2
3
4
5
6
7


KeyboardInterrupt: 

In [5]:
for line in responses:
    print(line)

{"Name_Of_Disease": "Diphtheria", "Type_Of_Microorganism": "Bacteria (Corynebacterium diphtheriae)", "Human_Only": "true", "Area_Of_Origin": "Nigeria", "Type_Of_Outbreak": "Multinational" }

{"Name_Of_Disease": "Carbon Monoxide Poisoning", "Type_Of_Microorganism": "Chemical", "Human_Only": "true", "Area_Of_Origin": "Algeria", "Type_Of_Outbreak": "Limited"}

{"Name_Of_Disease": "COVID-19", "Type_Of_Microorganism": "Virus (SARS-CoV-2)", "Human_Only": "true", "Area_Of_Origin": "Bangladesh", "Type_Of_Outbreak": "Limited"}

{"Name_Of_Disease":"Toad Toxicity", "Type_Of_Microorganism":null, "Human_Only":false, "Area_Of_Origin":null, "Type_Of_Outbreak":null}

{"Name_Of_Disease": "Yellow Fever", 
"Type_Of_Microorganism": "Virus (Flaviviridae)", 
"Human_Only": "false", 
"Area_Of_Origin": "South Sudan", 
"Type_Of_Outbreak": "Multinational"}

{"Name_Of_Disease": "Norovirus", "Type_Of_Microorganism": "Virus (RNA)", "Human_Only": "true", "Area_Of_Origin": "United States", "Type_Of_Outbreak": "Locali

In [ ]:
newFile = open("/home/team-2/RRC/data.txt", 'w')
for line in responses:
    newFile.write(line)
    print(line)
newFile.close()

In [6]:
readFile = open("/home/team-2/RRC/data.txt", 'r')
lines = readFile.readlines()

data = [json.loads(line) for line in lines]

selected_columns = ["Name_Of_Disease", "Type_Of_Microorganism", "Human_Only", "Area_Of_Origin", "Type_Of_Outbreak"]

df = pd.DataFrame(data, columns = selected_columns)
readFile.close()
df = df[df["Human_Only"] == "true"]
df = df[df["Type_Of_Outbreak"].isin(["National", "Multinational", "Pandemic", "Epidemic", "Outbreak"])]

df



,Name_Of_Disease,Type_Of_Microorganism,Human_Only,Area_Of_Origin,Type_Of_Outbreak
0,Diphtheria,Bacteria (Corynebacterium diphtheriae),true,Nigeria,Multinational
7,Lassa Fever,Bacteria (Rodenticulous),true,West Africa,Multinational
31,Norovirus,Virus(No),true,United States,Outbreak
32,Western Equine Encephalomyelitis,Virus(WEE),true,Argentina,Multinational
33,Diphtheria,Bacteria (Corynebacterium diphtheriae),true,Czech Republic,Epidemic
37,Cryptosporidiosis,Protozoa (Cryptosporidium spp.),true,Sweden,Multinational
38,Rabies,Virus (RABV),true,Worldwide,Pandemic
50,Measles,Virus (Rubeola virus),true,United States of America,Epidemic
56,Coronavirus Disease 2019 (COVID-19),Virus(SARS-CoV-2),true,"Wuhan, China",Pandemic
67,Poliomyelitis,Virus (Poliovirus),true,Africa,Multinational


In [7]:
with open("/home/team-2/RRC/SortedResults.txt", "w") as f:
    dfAsString = df.to_string(header=True, index=False)
    f.write(dfAsString)

In [8]:
df.drop('Human_Only', axis=1, inplace=True)

In [9]:
df

,Name_Of_Disease,Type_Of_Microorganism,Area_Of_Origin,Type_Of_Outbreak
0,Diphtheria,Bacteria (Corynebacterium diphtheriae),Nigeria,Multinational
7,Lassa Fever,Bacteria (Rodenticulous),West Africa,Multinational
31,Norovirus,Virus(No),United States,Outbreak
32,Western Equine Encephalomyelitis,Virus(WEE),Argentina,Multinational
33,Diphtheria,Bacteria (Corynebacterium diphtheriae),Czech Republic,Epidemic
37,Cryptosporidiosis,Protozoa (Cryptosporidium spp.),Sweden,Multinational
38,Rabies,Virus (RABV),Worldwide,Pandemic
50,Measles,Virus (Rubeola virus),United States of America,Epidemic
56,Coronavirus Disease 2019 (COVID-19),Virus(SARS-CoV-2),"Wuhan, China",Pandemic
67,Poliomyelitis,Virus (Poliovirus),Africa,Multinational


In [ ]:
business_units = []
index = 0
for pathogen in dfAsString:
    interested_business_unit  = medOllama('[INST] Given the Type_Of_Microorganism in {0}, return the key of what business unit(s) may be interesed ' +
    '[/INST] P&Gs 6 business units are: {{"FHC":"Relates to pathogens transmitted via contaminated surfaces, "' +
    '"PHC":"Upper respiratory tract infections, "' +
    '"OC": "Oral Cavity Pathogens & Upper respiratory tract infections, "' +
    '"PCC": "Pathogens transmitted via poor hand hygiene, "' +
    '"PGV": "Pathogens transmitted via insects/arthropods, "' +
    '"BFF": "Pathogens transmitted via bodily excrements and can linger in diapers or feminine pads"}}' + 
    ' [/INST Do not follow up with any text'.format(pathogen))
    business_units.append(interested_business_unit)
    index += 1
    print(index)
    time.sleep(.1)
    

In [ ]:
summaries = []
index = 0
for pathogen in dfAsString:
    summary = medOllama('[INST] Given the Name_Of_Disease in {0}, return a summary no more than 3 sentences of what the disease is and how it is affecting the modern world' +
                        '[/INST] This is going in a news letter so only include relevant information [/INST] Do not include any extra "User Questions')
    summaries.append[summary]
    time.sleep(.1)
    index += 1
    print(index)


In [ ]:
df["Interested_BU"] = business_units

In [10]:
print(tabulate(df, headers='keys', tablefmt='html', showindex=False))
with open("/home/team-2/Testing Data/PrettyTable.html", 'r') as file:
        content = file.read()
        
with open("/home/team-2/RRC/Table.html", "w") as f:
        f.write(content)
        f.write(tabulate(df, headers='keys', tablefmt='html', showindex=False))
        f.write("</body>\n</html>")



<table>
<thead>
<tr><th>Name_Of_Disease                    </th><th>Type_Of_Microorganism                 </th><th>Area_Of_Origin          </th><th>Type_Of_Outbreak  </th></tr>
</thead>
<tbody>
<tr><td>Diphtheria                         </td><td>Bacteria (Corynebacterium diphtheriae)</td><td>Nigeria                 </td><td>Multinational     </td></tr>
<tr><td>Lassa Fever                        </td><td>Bacteria (Rodenticulous)              </td><td>West Africa             </td><td>Multinational     </td></tr>
<tr><td>Norovirus                          </td><td>Virus(No)                             </td><td>United States           </td><td>Outbreak          </td></tr>
<tr><td>Western Equine Encephalomyelitis   </td><td>Virus(WEE)                            </td><td>Argentina               </td><td>Multinational     </td></tr>
<tr><td>Diphtheria                         </td><td>Bacteria (Corynebacterium diphtheriae)</td><td>Czech Republic          </td><td>Epidemic          </td></tr>
<